In [44]:
from src.helper_functions import xor_hex_strings, repeating_key_xor, xor_bytes, hamming_distance, output_repeated_block, has_repeated_blocks, find_block
from src.symmetric_encryption import is_ecb_mode
import src.aes128 as aes128
import src.convert as convert
import src.padding as padding
# 
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import secrets

## **Challenge 17: PKCS#7 padding validation**

In [45]:
# Assume that key is unknown but same for all encryptions
encryption_oracle_17_key = secrets.token_bytes(16)

In [46]:
def encryption_oracle_17() -> tuple[bytes, bytes]:
    plaintext_list = [
        b"MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=",
        b"MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=",
        b"MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==",
        b"MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==",
        b"MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl",
        b"MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==",
        b"MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==",
        b"MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=",
        b"MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=",
        b"MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93",
    ]

    chosen_plaintext = secrets.choice(plaintext_list)
    chosen_plaintext = base64.b64decode(chosen_plaintext)
    chosen_plaintext = padding.apply_pkcs_7(chosen_plaintext, aes128.AES128_KEY_LENGTH)
    
    IV = secrets.token_bytes(16)
    return aes128.cbc_encrypt(chosen_plaintext, encryption_oracle_17_key, IV), IV


def correct_padding_oracle(ciphertext: bytes, IV: bytes) -> bool:
    plaintext = aes128.cbc_decrypt(ciphertext, encryption_oracle_17_key, IV)
    try:
        padding.remove_pkcs_7(plaintext)
        return True
    except Exception:
        return False

In [47]:
cyphertext, IV = encryption_oracle_17()
correct_padding_oracle(cyphertext, IV)

True

In [48]:
cyphertext_blocks = [IV] + convert.bytes2blocks(cyphertext, aes128.AES128_KEY_LENGTH)
cyphertext_blocks

[b"\x0c`W\x9e\xb2\xf4\x98\xe0\xa4'i\x10\x81\xd2\xe5\xb5",
 b'\xa6\x86\x1e\x1e\n\x02\xa8g\x10!8Q\xc1uM?',
 b'y1+5\xc7<\x1bU\xad\\xiM\xaf:\x9a']

In [70]:

# 1 * convert.int2bytes(1)
plaintext_block = aes128.AES128_KEY_LENGTH * convert.int2bytes(0)

zeros_IV = aes128.AES128_KEY_LENGTH * convert.int2bytes(0)

for i in range(256):
    plaintext_block = 15 * convert.int2bytes(0) + convert.int2bytes(i)
    padding_block = padding.apply_pkcs_7(convert.int2bytes(0) * (aes128.AES128_KEY_LENGTH - 1), aes128.AES128_KEY_LENGTH)
    fake_IV = xor_bytes(plaintext_block, xor_bytes(padding_block, cyphertext_blocks[1]))

    zeroing_IV = xor_bytes(padding_block, cyphertext_blocks[1])

    if correct_padding_oracle(cyphertext_blocks[1], fake_IV):
        print(i)





233
